In [0]:
from pyspark.sql.functions import *

In [0]:
%run "/Workspace/Shared/Optum_Project/connectors"

In [0]:
adlsconnector()
Hospital_df = readsilverdata("Hospital_S.csv")
Patient_df = readsilverdata("Patient_S.csv")
Claims_df = readsilverdata("Claims_S.csv")
Disease_df = readsilverdata("Disease_S.csv")
Group_df = readsilverdata("Group_S.csv")
SubGroup_df = readsilverdata("Subgroup_S.csv")
Subscriber_df = readsilverdata("Subscriber_S.csv")

adls connection successful
adls connection successful
adls connection successful
adls connection successful
adls connection successful
adls connection successful
adls connection successful
adls connection successful


In [0]:
Hospital_df = Hospital_df.select(
    col("Hospital_id"),col("Hospital_name"),
    col("city").alias("Hospital_city"),
    col("state").alias("Hospital_state"),
    col("country").alias("Hospital_country")
)


In [0]:
SubGroup_df = SubGroup_df.select(
    col("subgrp_sk"),
    col("subgrp_name"),
    col("monthly_premium"),
    col("subgrp_id").alias("grp_id")
)


In [0]:
Subscriber_df = Subscriber_df.withColumnRenamed('Subgrp_id','Patient_Policy_Subgrp_id')

In [0]:
Group_df = Group_df.select(col("country").alias("policy_origin_country"),
                           col("premium_written").alias("policy_origin_premium_written"),
                           col("zip_code").alias("policy_origin_zip_code"),
                           col("grp_id"),
                           col("grp_name").alias("policy_origin_grp_name"),
                           col("grp_type").alias("policy_origin_grp_type"),
                           col("city").alias("policy_origin_city"))

In [0]:
merged_df = Claims_df.join(Patient_df, Claims_df.patient_id == Patient_df.Patient_id, "left").drop(Patient_df.Patient_id)\
                    .join(Subscriber_df, Claims_df.SUB_ID == Subscriber_df.sub_id, "left").drop(Subscriber_df.sub_id, Subscriber_df.age,Subscriber_df.City)\
                    .join(Hospital_df, Patient_df.hospital_id == Hospital_df.Hospital_id, "left").drop(Hospital_df.Hospital_id)\
                    .join(SubGroup_df, Subscriber_df.Patient_Policy_Subgrp_id == SubGroup_df.subgrp_sk, "left").drop(SubGroup_df.subgrp_sk)\
                    .join(Disease_df, Claims_df.disease_name == Disease_df.disease_name, "left").drop(Disease_df.disease_name).withColumnRenamed('subgrp_id','Patient_Claimed_Subgrp_id')\
                    .join(Group_df, SubGroup_df.grp_id == Group_df.grp_id, "left").drop(Group_df.grp_id)
display(merged_df.head(100))

Claim_Or_Rejected,SUB_ID,claim_amount,claim_date,claim_id,claim_type,disease_name,patient_id,Patient_name,patient_gender,disease_name,city,hospital_id,patient_age,first_name,last_name,Street,Gender,Country,Zip Code,Patient_Policy_Subgrp_id,Elig_ind,eff_date,term_date,Hospital_name,Hospital_city,Hospital_state,Hospital_country,subgrp_name,monthly_premium,grp_id,Patient_Claimed_Subgrp_id,disease_id,policy_origin_country,policy_origin_premium_written,policy_origin_zip_code,policy_origin_grp_name,policy_origin_grp_type,policy_origin_city
N,SUBID1000,79874,1949-03-14,0,claims of value,Galactosemia,187158,Harbir,Female,Galactosemia,Rourkela,H1001,101,null,null,null,null,null,null,null,null,null,null,Medanta The Medicity,Gurgaon,Haryana,India,null,null,null,S109,110052,null,null,null,null,null,null
N,SUBID10001,151142,1970-03-16,1,claims of policy,Bladder cancer,112766,Brahmdev,Female,Bladder cancer,Tiruvottiyur,H1016,76,Brahmdev,Sonkar,Lala Marg,Female,India,34639,S105,Y,1968-12-20,1970-05-16,Jaslok Hospital and Research Centre,Mumbai,Maharashtra,India,Allergies,2300,GRP124,S107,110039,India,81000,482033,Edelweiss Tokio Life Insurance Co. Ltd.,Private,Mumbai
N,SUBID10001,151142,1970-03-16,1,claims of policy,Bladder cancer,112766,Brahmdev,Female,Bladder cancer,Tiruvottiyur,H1016,76,Brahmdev,Sonkar,Lala Marg,Female,India,34639,S105,Y,1968-12-20,1970-05-16,Jaslok Hospital and Research Centre,Mumbai,Maharashtra,India,Allergies,2300,GRP114,S107,110039,India,33000,482043,Sahara India Life Insurance Co. Ltd.,Private,Lucknow
N,SUBID10001,151142,1970-03-16,1,claims of policy,Bladder cancer,112766,Brahmdev,Female,Bladder cancer,Tiruvottiyur,H1016,76,Brahmdev,Sonkar,Lala Marg,Female,India,34639,S105,Y,1968-12-20,1970-05-16,Jaslok Hospital and Research Centre,Mumbai,Maharashtra,India,Allergies,2300,GRP104,S107,110039,India,59000,482028,ICICI Prudential Life Insurance Co. Ltd.,Private,Mumbai
N,SUBID10001,151142,1970-03-16,1,claims of policy,Bladder cancer,112766,Brahmdev,Female,Bladder cancer,Tiruvottiyur,H1016,76,Brahmdev,Sonkar,Lala Marg,Female,India,34639,S105,Y,1968-12-20,1970-05-16,Jaslok Hospital and Research Centre,Mumbai,Maharashtra,India,Allergies,2300,GRP153,S107,110039,India,86000,482004,Shriram General Insurance,Private,Jaipur
N,SUBID10002,59924,2008-02-03,2,claims of value,Kidney cancer,199252,Ujjawal,Male,Kidney cancer,Berhampur,H1009,45,Ujjawal,Devi,Mammen Zila,Male,India,914455,S106,N,2000-04-16,2008-05-04,Indraprastha Apollo Hospital,Delhi,UT,India,Self inflicted,1200,GRP157,S107,110037,India,92000,482019,United India Insurance Company,Govt.,Chennai
N,SUBID10002,59924,2008-02-03,2,claims of value,Kidney cancer,199252,Ujjawal,Male,Kidney cancer,Berhampur,H1009,45,Ujjawal,Devi,Mammen Zila,Male,India,914455,S106,N,2000-04-16,2008-05-04,Indraprastha Apollo Hospital,Delhi,UT,India,Self inflicted,1200,GRP147,S107,110037,India,99000,482040,Raheja QBE General Insurance,Private,Mumbai
N,SUBID10002,59924,2008-02-03,2,claims of value,Kidney cancer,199252,Ujjawal,Male,Kidney cancer,Berhampur,H1009,45,Ujjawal,Devi,Mammen Zila,Male,India,914455,S106,N,2000-04-16,2008-05-04,Indraprastha Apollo Hospital,Delhi,UT,India,Self inflicted,1200,GRP137,S107,110037,India,47000,482015,Future Generali India Insurance,Private,Mumbai
N,SUBID10002,59924,2008-02-03,2,claims of value,Kidney cancer,199252,Ujjawal,Male,Kidney cancer,Berhampur,H1009,45,Ujjawal,Devi,Mammen Zila,Male,India,914455,S106,N,2000-04-16,2008-05-04,Indraprastha Apollo Hospital,Delhi,UT,India,Self inflicted,1200,GRP127,S107,110037,India,60000,482020,Agriculture Insurance Company of India,Govt.,New Delhi
N,SUBID10002,59924,2008-02-03,2,claims of value,Kidney cancer,199252,Ujjawal,Male,Kidney cancer,Berhampur,H1009,45,Ujjawal,Devi,Mammen Zila,Male,India,914455,S106,N,2000-04-16,2008-05-04,Indraprastha Apollo Hospital,Delhi,UT,India,Self inflicted,1200,GRP117,S107,110037,India,59000,482017,Future Generali India Life Insurance Co. Ltd.,Private,Mumbai


In [0]:
write2gold(merged_df,"Optum_G.csv")

CSV file successfully written to abfss://optum@optumadlsstrg.dfs.core.windows.net/gold//Optum_G.csv


'abfss://optum@optumadlsstrg.dfs.core.windows.net/gold//Optum_G.csv'